####License

In [ ]:
#Copyright 2024 Google LLC

#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at

#    https://www.apache.org/licenses/LICENSE-2.0

#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

#Google Colab GA4 Easy Datamart Preparation
![Placeholder Image](https://drive.google.com/uc?export=view&id=1eNinLEkFUGqT8M98C-3mhGYzngX141CG)

This Google Colab notebook facilitates the creation of a user-level datamart using Google Analytics 4 (GA4) data. The process involves building a SQL query to be executed in BigQuery, resulting in a customized datamart for analysis. The script utilizes various parameters provided by the user to tailor the datamart creation process. Recommended if you have sizeable logged in user data.


---

**Code Execution:**

The script begins by authenticating the user and initializing the BigQuery client. It then proceeds to format the provided time window and constructs a user mapping table query to associate user pseudo-IDs with user IDs. This mapping is crucial for attributing past behavior to a unified user ID.

The user ID input is processed based on the provided alias, allowing flexibility in handling user identification. The resulting SQL query is saved to a file for reference.

Custom events for predictive audiences are then processed, allowing users to specify events for inclusion in the datamart. The SQL queries are dynamically adjusted based on user input.

Finally, the user mapping and datamart queries are executed in BigQuery, creating the desired user-level datamart for analysis.
The resulting datamart can be used for user level measurement and feature engineering in ML applications. Includes built in RFM+ metrics (user engagement, purchase behavior).

The output data dictionary: [link](https://docs.google.com/spreadsheets/d/12FJzkwUl3gEjMNgvtLzqLKOsCxKy4Mog-8AYbuEXPdc/edit?pli=1&resourcekey=0-b1BaOperI3qQlrlQh29wIg#gid=601097648)

Detailed implementation doc: [link](https://docs.google.com/document/d/1WTVFEOfacX3flo8eExh3J6gyKVEOpItN_WKC5RMehYI/edit?resourcekey=0-SavOE28uPRruGefuKDrthQ&tab=t.0#heading=h.6mwpcxb2gxe9)

---



Note: Ensure proper authentication and authorization for GCP and BigQuery to execute the script successfully.

In [1]:
#@title Inputs
GCP_Job_project_name = "turkey-analytical-projects" #@param {type:"string"}
GCP_Job_region = "EU" #@param {type:"string"}
Big_query_project_name = "arcelik-bi-project" #@param {type:"string"}
Data_set_name = "goog123" #@param {type:"string"}
GA4_table_name = "analytics_294368252" #@param {type:"string"}
User_id_alias = "user_id" #@param {type:"string"}
App_stream_to_ga4 = False #@param {type:"boolean"}
Have_Ga4_custom_audiences = False #@param {type:"boolean"}
Ga4_custom_audience_event_names = "" #@param {type:"string"}
Datamart_start_date= "2024-11-01" #@param {type:"date"}
Datamart_end_date= "2024-11-03" #@param {type:"date"}

*Fill the fields with desired parameters to build the userdatamart:*




* **GCP_Job_project_name:** Google Cloud Platform (GCP) project name for job execution.


* **GCP_Job_region:** GCP region for job execution. (make sure that the dataset and events tables are at the same region)


* **Big_query_project_name:** Project name in BigQuery where the datamart will be created


* **Data_set_name:** Name of the dataset within the specified BigQuery project. Tables will be created under this dataset.


* **GA4_table_name:** Name of the GA4 table containing the events data. Exp: analytics_12345


* **User_id_alias:** Alias for user identification, supporting nested properties and direct user ID references.If the user_id is recorder in user_id column in GA4 data just keep it as "user_id". If the user_id is recorded under user_properties parameters of GA4, make the value: "user_properties, user_id, string_value" comma delimited.


* **App_stream_to_ga4:** Boolean indicating whether app streaming data should be included.


* **Have_Ga4_custom_audiences:** Boolean indicating whether custom GA4 audiences are to be included.Check the box if you have pre-built custom/predictive audiences and want to export it to user datamart.


* **Ga4_custom_audience_event_names:** Names of custom events triggering GA4 custom audiences. If you don't have custom audiences, make sure you unchecked the previous box. Can accept up to 3 event triggers delimited by comma. Exp: likely_sevenday_purchasers, first_time_sevenday_purchasers, likely_sevenday_churning


* **Datamart_start_date:** Start date for the datamart time window.


* **Datamart_end_date:** End date for the datamart time window.

## Authenticate

In [2]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import numpy as np
import pandas as pd
import pandas_gbq
import datetime
project = GCP_Job_project_name # Project ID to run the job
location = GCP_Job_region # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

##Build GA4 user datamart

###Build user_id mapping table

In [3]:
#create the dataset
def create_dataset(project_id, dataset_id, location):

  # Create the dataset reference
  dataset_ref = client.dataset(dataset_id, project=project_id)

  # Specify the dataset properties (optional)
  dataset = bigquery.Dataset(dataset_ref)
  dataset.location = location  # Change the location as needed

  # Create the dataset
  client.create_dataset(dataset, exists_ok=True)
  print(f"Dataset {dataset_id} created.")


create_dataset(project_id=Big_query_project_name, dataset_id=Data_set_name, location=location)


# time formatting for query
last_date_sql = """CAST('last_date' AS DATE)""".replace("last_date", Datamart_end_date)

def reformat_date(date):
  date = date.replace("-", "")
  return date

table_suffix_start_date = reformat_date(Datamart_start_date)
table_suffix_end_date = reformat_date(Datamart_end_date)
time_frame = f"(_TABLE_SUFFIX BETWEEN '{table_suffix_start_date}' AND '{table_suffix_end_date}')"

# create the user_id mapping table query

user_id_mapping_query= f"""

create or replace table `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as
(
with u0 as (
select
user_pseudo_id,
user_id_place_holder as fixed_user_id,
event_date
from `{Big_query_project_name}.{GA4_table_name}.events_*`
-- where {time_frame}
)

---- user id user_pseudo_id mapping using last date events

select
* except(rn)
from
(
select
user_pseudo_id,
fixed_user_id as user_id,
row_number() over (partition by user_pseudo_id order by event_date desc) as rn
from u0
where fixed_user_id is not null
)
where rn = 1
);


"""


def user_id_input_fix(User_id_alias, user_id_mapping_query):
  if "user_properties" in User_id_alias:
    print("user_id is nested in user_properties column in ga4")
    user_properties,field_name,value_type = User_id_alias.split(",")
    field_name = field_name.strip()
    value_type = value_type.strip()

    user_id_snippet= """(select value.value_type from unnest(user_properties) where key='field_name')""".replace("field_name", field_name)
    user_id_snippet= user_id_snippet.replace("value_type", value_type)

    user_id_mapping_query = user_id_mapping_query.replace("user_id_place_holder", user_id_snippet)
    print(user_id_mapping_query)
    return user_id_mapping_query

  elif "user_id" in User_id_alias:
    print("user_id in user_id column in ga4")
    user_id_mapping_query = user_id_mapping_query.replace("user_id_place_holder", "user_id")
    print(user_id_mapping_query)
    return user_id_mapping_query

  else:
    print("Re check user_id alisa input! Make sure that it is seperated with comas")


user_id_mapping_query= user_id_input_fix(User_id_alias, user_id_mapping_query)

# Specify the file name
file_name = "user_id_mapping_query.sql"  # You can use .sql extension to indicate that it is an SQL file

# Write the SQL query to the file
with open(file_name, "w") as file:
    file.write(user_id_mapping_query)

print(f"The SQL query has been saved to {file_name}.")

Dataset goog123 created.
user_id in user_id column in ga4


create or replace table `arcelik-bi-project.goog123.user_mapping_table` as
(
with u0 as (
select
user_pseudo_id,
user_id as fixed_user_id,
event_date
from `arcelik-bi-project.analytics_294368252.events_*`
-- where (_TABLE_SUFFIX BETWEEN '20241101' AND '20241103')
)

---- user id user_pseudo_id mapping using last date events

select
* except(rn)
from
(
select
user_pseudo_id,
fixed_user_id as user_id,
row_number() over (partition by user_pseudo_id order by event_date desc) as rn
from u0
where fixed_user_id is not null
)
where rn = 1
);



The SQL query has been saved to user_id_mapping_query.sql.


###Build user datamart query

custom audience triggers query

In [4]:
#adjust the section for ga4 predictive audiences query
pred_query ="""max(case when event_name = 'likely_sevenday_purchasers' then 1 else 0 end) as likely_sevenday_purchasers,
max(case when event_name = 'first_time_sevenday_purchasers' then 1 else 0 end) as first_time_sevenday_purchasers,
max(case when event_name = 'likely_sevenday_churning' then 1 else 0 end) as likely_sevenday_churning
"""

null_adjust_pred_aud= """IFNULL(likely_sevenday_purchasers, 0) as likely_sevenday_purchasers,
IFNULL(first_time_sevenday_purchasers, 0) as first_time_sevenday_purchasers,
IFNULL(likely_sevenday_churning, 0) as likely_sevenday_churning,"""


#get the input and clean it
pred_events=[event.strip() for event in Ga4_custom_audience_event_names.split(",")]

if len(pred_events) == 1:
  pred_query=pred_query.replace(pred_query,f"max(case when event_name = '{pred_events[0]}' then 1 else 0 end) as {pred_events[0]}")
  null_adjust_pred_aud=null_adjust_pred_aud.replace(null_adjust_pred_aud,f"""IFNULL({pred_events[0]}, 0) as {pred_events[0]},""")

elif len(pred_events) == 2:
  pred_query=pred_query.replace(pred_query,f"""max(case when event_name = '{pred_events[0]}' then 1 else 0 end) as {pred_events[0]},
max(case when event_name = '{pred_events[1]}' then 1 else 0 end) as {pred_events[1]}""")
  null_adjust_pred_aud=null_adjust_pred_aud.replace(null_adjust_pred_aud,f"""IFNULL({pred_events[0]}, 0) as {pred_events[0]},
IFNULL({pred_events[1]}, 0) as {pred_events[1]},""")

elif len(pred_events) == 3:
  pred_query=pred_query.replace(pred_query,f"""max(case when event_name = '{pred_events[0]}' then 1 else 0 end) as {pred_events[0]},
max(case when event_name = '{pred_events[1]}' then 1 else 0 end) as {pred_events[1]},
max(case when event_name = '{pred_events[2]}' then 1 else 0 end) as {pred_events[2]}""")
  null_adjust_pred_aud=null_adjust_pred_aud.replace(null_adjust_pred_aud,f"""IFNULL({pred_events[0]}, 0) as {pred_events[0]},
IFNULL({pred_events[1]}, 0) as {pred_events[1]},
IFNULL({pred_events[2]}, 0) as {pred_events[2]},""")

elif len(pred_events) > 3:
  print("too much events")
  Have_Ga4_custom_audiences=False

print(pred_events)
print(len(pred_events))
print(pred_query)
print(null_adjust_pred_aud)





['']
1
max(case when event_name = '' then 1 else 0 end) as 
IFNULL(, 0) as ,


In [5]:
user_datamart_query= f"""
create or replace table `{Big_query_project_name}.{Data_set_name}.user_id_data_ga4_{table_suffix_start_date}_{table_suffix_end_date}` as
--- geo, device info, user_ltv —
-- for each user assign the latest observed device
with t1 as(
select
* except(RN)
from
 (SELECT
 u1.user_id,


 geo.continent,
 geo.sub_continent,
 geo.region,
 geo.country,
 geo.city,


 device.category,
 device.mobile_brand_name,
 device.operating_system,


 user_first_touch_timestamp,
 user_ltv.revenue as user_ltv,


 row_number() over (partition by u1.user_id order by event_date desc) as RN


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and u1.user_id is not null
   )
where RN = 1
)
,


-- last events
t2 as (
 select * except(RN)
 from
   (SELECT
   u1.user_id,
   event_date as last_event_date,
   event_name as last_event_name,
   IFNULL(traffic_source.name, 'NOTSET') as last_traffic_source_name,
   IFNULL(traffic_source.medium,'NOTSET') as last_traffic_source_medium,
   IFNULL(traffic_source.`source`, 'NOTSET') as last_traffic_source_source,
   row_number() over (partition by u1.user_id order by event_date desc) as RN
   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and u1.user_id is not null    )
 where RN = 1
)
,
-- join tables
p1 as (
select
t1.*,
t2.*  except(user_id)
from t1
left join t2 on t1.user_id = t2.user_id
)
,


-- total pageviews and the distinct days page viewed (visit frequency)


pw as (
select
u1.user_id,


sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then 1 else 0 end) ) as number_of_page_views_7D,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then 1 else 0 end) ) as number_of_page_views_14D,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then 1 else 0 end) ) as number_of_page_views_30D,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then 1 else 0 end) ) as number_of_page_views_60D,


count(distinct (case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then event_date else null end) ) as visit_freq_7D,
count(distinct (case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then event_date else null end) ) as visit_freq_14D,
count(distinct (case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then event_date else null end) ) as visit_freq_30D,
count(distinct (case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then event_date else null end) ) as visit_freq_60D


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and event_name = 'page_view' and u1.user_id is not null
   group by 1

),


p2 as(
select
p1.*,
pw.* except(user_id)
from p1
left join pw on p1.user_id=pw.user_id
)
,


-- visit durations
vd as (
select
u1.user_id,


sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then ep.value.int_value else 0 end) )/1000 as visit_duration_7Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then ep.value.int_value else 0 end) )/1000 as visit_duration_14Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then ep.value.int_value else 0 end) )/1000 as visit_duration_30Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then ep.value.int_value else 0 end) )/1000 as visit_duration_60Ds


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x, unnest(event_params) as ep
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and event_name = 'page_view' and ep.key ='engagement_time_msec' and u1.user_id is not null
   group by 1
)
,


-- screen view and screen view durations (use if app is also linked with GA4)
sw as (
select
u1.user_id,


sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then 1 else 0 end) ) as number_of_screen_views_7D,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then 1 else 0 end) ) as number_of_screen_views_14D,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then 1 else 0 end) ) as number_of_screen_views_30D,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then 1 else 0 end) ) as number_of_screen_views_60D


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and event_name = 'screen_view' and u1.user_id is not null
   group by 1
)
,


sd as (
select
u1.user_id,

sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_7Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_14Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_30Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_60Ds


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x , unnest(event_params) as ep
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and event_name = 'screen_view' and ep.key ='engagement_time_msec' and u1.user_id is not null
   group by 1
)
,


sw2 as (
select
sw.*,
sd.* except(user_id)
from sw
left join sd on sw.user_id = sd.user_id
)
,
-- join tables


p3 as (
select
p2.*,
vd.* except(user_id),
sw2.* except(user_id)
from p2
left join vd on p2.user_id = vd.user_id
left join sw2 on p2.user_id = sw2.user_id
)
,

--- purchases---

pur as(


 select
 u1.user_id,

 round(sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then itm.price_in_usd else 0 end),2) as total_revenue_7D,
 round(sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then itm.price_in_usd else 0 end),2) as total_revenue_14D,
 round(sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then itm.price_in_usd else 0 end),2) as total_revenue_30D,
 round(sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then itm.price_in_usd else 0 end),2) as total_revenue_60D,

 date_diff({last_date_sql} , max(PARSE_DATE("%Y%m%d", event_date)), DAY) as purchase_recency,

 sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then 1 else 0 end) as purchase_events_60D,
 sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then 1 else 0 end) as purchase_events_30D,
 sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then 1 else 0 end) as purchase_events_14D,
 sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then 1 else 0 end) as purchase_events_7D


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x , unnest(items) as itm
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id and u1.user_id is not null
   where {time_frame} and event_name = 'purchase'
   group by 1 )
,


-- join tables
p4 as (
select
p3.*,
pur.* except(user_id)
from
p3
left join pur on p3.user_id = pur.user_id
)
,


-- total clicks --
c1 as (
select
u1.user_id,
sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then 1 else 0 end) as total_clicks_7D,
sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then 1 else 0 end) as total_clicks_14D,
sum(case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then 1 else 0 end) as total_clicks_30D

   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and event_name = 'click' and u1.user_id is not null
   group by 1
)
,


-- join tables
p5 as (
select
p4.*,
c1.* except(user_id)
from
p4
left join c1 on p4.user_id = c1.user_id
)
,


-- predictive metrics
pred as
(
SELECT
u1.user_id,

{pred_query}

FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on x.user_pseudo_id = u1.user_pseudo_id where {time_frame} and u1.user_id is not null
group by 1
)
,

-- join tables

p6 as (
select
p5.*,
pred.* except(user_id)
from
p5
left join pred on p5.user_id = pred.user_id
)

-- NULL HANDLING --
select
user_id,
continent,
sub_continent,
region,
country,
city,
category,
mobile_brand_name,
operating_system,
user_first_touch_timestamp,
last_event_date,
last_event_name,
last_traffic_source_name,
last_traffic_source_medium,
last_traffic_source_source,
IFNULL(user_ltv, 0) as user_ltv,
IFNULL(number_of_page_views_7D, 0) as number_of_page_views_7D,
IFNULL(number_of_page_views_14D, 0) as number_of_page_views_14D,
IFNULL(number_of_page_views_30D, 0) as number_of_page_views_30D,
IFNULL(number_of_page_views_60D, 0) as number_of_page_views_60D,
IFNULL(visit_freq_7D, 0) as visit_freq_7D,
IFNULL(visit_freq_14D, 0) as visit_freq_14D,
IFNULL(visit_freq_30D, 0) as visit_freq_30D,
IFNULL(visit_freq_60D, 0) as visit_freq_60D,
IFNULL(visit_duration_7Ds, 0) as visit_duration_7Ds,
IFNULL(visit_duration_14Ds, 0) as visit_duration_14Ds,
IFNULL(visit_duration_30Ds, 0) as visit_duration_30Ds,
IFNULL(visit_duration_60Ds, 0) as visit_duration_60Ds,
IFNULL(number_of_screen_views_7D, 0) as number_of_screen_views_7D,
IFNULL(number_of_screen_views_14D, 0) as number_of_screen_views_14D,
IFNULL(number_of_screen_views_30D, 0) as number_of_screen_views_30D,
IFNULL(number_of_screen_views_60D, 0) as number_of_screen_views_60D,
IFNULL(screen_view_duration_7Ds, 0) as screen_view_duration_7Ds,
IFNULL(screen_view_duration_14Ds, 0) as screen_view_duration_14Ds,
IFNULL(screen_view_duration_30Ds, 0) as screen_view_duration_30Ds,
IFNULL(screen_view_duration_60Ds, 0) as screen_view_duration_60Ds,
IFNULL(total_revenue_7D, 0) as total_revenue_7D,
IFNULL(total_revenue_14D, 0) as total_revenue_14D,
IFNULL(total_revenue_30D, 0) as total_revenue_30D,
IFNULL(total_revenue_60D, 0) as total_revenue_60D,
IFNULL(purchase_recency, 0) as purchase_recency,
IFNULL(purchase_events_60D, 0) as purchase_events_60D,
IFNULL(purchase_events_30D, 0) as purchase_events_30D,
IFNULL(purchase_events_14D, 0) as purchase_events_14D,
IFNULL(purchase_events_7D, 0) as purchase_events_7D,
IFNULL(total_clicks_7D, 0) as total_clicks_7D,
IFNULL(total_clicks_14D, 0) as total_clicks_14D,
IFNULL(total_clicks_30D, 0) as total_clicks_30D,
{null_adjust_pred_aud}
current_date() as crtdate
from p6
;

"""

In [6]:
if App_stream_to_ga4 == False:
  remove_code_part1 = f"""sd as (
select
u1.user_id,

sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 8 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_7Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 15 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_14Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 31 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_30Ds,
sum ((case when PARSE_DATE("%Y%m%d", event_date) > {last_date_sql}- 61 then ep.value.int_value else 0 end) )/1000 as screen_view_duration_60Ds


   FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x , unnest(event_params) as ep
   left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where {time_frame} and event_name = 'screen_view' and ep.key ='engagement_time_msec' and u1.user_id is not null
   group by 1
)
,


sw2 as (
select
sw.*,
sd.* except(user_id)
from sw
left join sd on sw.user_id = sd.user_id
)
,"""

  remove_code_part2 = """,
sw2.* except(user_id)"""

  remove_code_part3 ="""left join sw2 on p2.user_id = sw2.user_id"""

  remove_code_part4 ="""IFNULL(number_of_screen_views_7D, 0) as number_of_screen_views_7D,
IFNULL(number_of_screen_views_14D, 0) as number_of_screen_views_14D,
IFNULL(number_of_screen_views_30D, 0) as number_of_screen_views_30D,
IFNULL(number_of_screen_views_60D, 0) as number_of_screen_views_60D,
IFNULL(screen_view_duration_7Ds, 0) as screen_view_duration_7Ds,
IFNULL(screen_view_duration_14Ds, 0) as screen_view_duration_14Ds,
IFNULL(screen_view_duration_30Ds, 0) as screen_view_duration_30Ds,
IFNULL(screen_view_duration_60Ds, 0) as screen_view_duration_60Ds,"""

  user_datamart_query = user_datamart_query.replace(remove_code_part1, "")
  user_datamart_query = user_datamart_query.replace(remove_code_part2, "")
  user_datamart_query = user_datamart_query.replace(remove_code_part3, "")
  user_datamart_query = user_datamart_query.replace(remove_code_part4, "")


In [7]:
if Have_Ga4_custom_audiences == False:
  remove_code_part5 = f"""-- predictive metrics
pred as
(
SELECT
u1.user_id,

max(case when event_name = 'likely_sevenday_purchasers' then 1 else 0 end) as likely_sevenday_purchasers,
max(case when event_name = 'first_time_sevenday_purchasers' then 1 else 0 end) as first_time_sevenday_purchasers,
max(case when event_name = 'likely_sevenday_churning' then 1 else 0 end) as likely_sevenday_churning

FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on x.user_pseudo_id = u1.user_pseudo_id where {time_frame} and u1.user_id is not null
group by 1
)
,"""

  remove_code_part6 = """,
pred.* except(user_id)"""

  remove_code_part7 ="""left join pred on p5.user_id = pred.user_id"""

  remove_code_part8 ="""IFNULL(likely_sevenday_purchasers, 0) as likely_sevenday_purchasers,
IFNULL(first_time_sevenday_purchasers, 0) as first_time_sevenday_purchasers,
IFNULL(likely_sevenday_churning, 0) as likely_sevenday_churning,"""

  user_datamart_query = user_datamart_query.replace(remove_code_part5, "")
  user_datamart_query = user_datamart_query.replace(remove_code_part6, "")
  user_datamart_query = user_datamart_query.replace(remove_code_part7, "")
  user_datamart_query = user_datamart_query.replace(remove_code_part8, "")





In [8]:
if Have_Ga4_custom_audiences == False:
  user_datamart_query=user_datamart_query.replace(pred_query,"")
  user_datamart_query=user_datamart_query.replace(null_adjust_pred_aud,"")

In [9]:
  print(user_datamart_query)


create or replace table `arcelik-bi-project.goog123.user_id_data_ga4_20241101_20241103` as
--- geo, device info, user_ltv —
-- for each user assign the latest observed device
with t1 as(
select
* except(RN)
from
 (SELECT
 u1.user_id,


 geo.continent,
 geo.sub_continent,
 geo.region,
 geo.country,
 geo.city,


 device.category,
 device.mobile_brand_name,
 device.operating_system,


 user_first_touch_timestamp,
 user_ltv.revenue as user_ltv,


 row_number() over (partition by u1.user_id order by event_date desc) as RN


   FROM `arcelik-bi-project.analytics_294368252.events_*` as x
   left join `arcelik-bi-project.goog123.user_mapping_table` as u1 on u1.user_pseudo_id = x.user_pseudo_id
   where (_TABLE_SUFFIX BETWEEN '20241101' AND '20241103') and u1.user_id is not null
   )
where RN = 1
)
,


-- last events
t2 as (
 select * except(RN)
 from
   (SELECT
   u1.user_id,
   event_date as last_event_date,
   event_name as last_event_name,
   IFNULL(traffic_source.name, 'NOTSET') as last_t

In [10]:
# Specify the file name
file_name = "user_datamart_query.sql"  # You can use .sql extension to indicate that it is an SQL file

# Write the SQL query to the file
with open(file_name, "w") as file:
    file.write(user_datamart_query)

### Create tables in Bigquery

In [11]:
queryjob1 =client.query(user_id_mapping_query)
queryjob1.result()
queryjob2 = client.query(user_datamart_query)
queryjob2.result()

#Join with CRM Data

*Make sure you have a hashed email column ready in your CRM table for activation*

In [ ]:
#@title CRM-GA4 Inputs
Use_previously_built_datamart_in_first_stage = True #@param {type:"boolean"}

if Use_previously_built_datamart_in_first_stage:
  GA4_datamart_table_name = f"{Big_query_project_name}.{Data_set_name}.user_id_data_ga4_{table_suffix_start_date}_{table_suffix_end_date}"

else:
  GA4_datamart_table_name = input("Enter GA4 Datamart Table Name (format=project_name.dataset_name.table_name): ") or f"{Big_query_project_name}.{Data_set_name}.user_id_data_ga4_{table_suffix_start_date}_{table_suffix_end_date}"

#GA4_datamart_table_name = GA4_datamart_full_name #@param {type:"string"}
CRM_table_full_name = "Your_project_name.your_dataset_name.crm_table_name" #@param {type:"string"}
CRM_user_id_alias = "user_id_column_name" #@param {type:"string"}
Output_table_name = "ota_ga4_joined" #@param {type:"string"}

* **Use_previously_built_datamart_in_first_stage:**
The table generated during the build datamart stage is the default table for joining with CRM data. If you wish to join with a different table, such as the scored output table from a prediction model, deselect the checkbox and utilize the specified input section. Input format exp: project_name.dataset_name.table_name

* **CRM_table_full_name:** Make sure you write the name in the following format: your_projectname.dataset_name.crm_table_name. Your CRM table's dataset should be in the same region with the GA4 dataset for the join.

* **CRM_user_id_alias:** the name of the user id field. This user id will be used as match key to join with GA4 data.

* **Output_table_name:** Rename output final table that includes CRM and GA4 data

### Generate the CRM - GA4 join query

In [ ]:
crm_ga4_join_query = f"""
create or replace table `{Big_query_project_name}.{Data_set_name}.{Output_table_name}` as
SELECT
*
FROM `{CRM_table_full_name}` t1
left join `{GA4_datamart_table_name}` t2 on cast(t1.{CRM_user_id_alias} as string)= cast(t2.user_id as string)
;"""

# Specify the file name
query_name = "crm_ga4_join_query.sql"  # You can use .sql extension to indicate that it is an SQL file

# Write the SQL query to the file
with open(query_name, "w") as file:
    file.write(crm_ga4_join_query)

### Create tables in Bigquery

In [ ]:
queryjob3 = client.query(crm_ga4_join_query)
queryjob3.result()

# Experiment with Modeling

##Event Propensity

![Example Image](https://drive.google.com/uc?export=view&id=1cc4mVGgCyayjX4m4FCETUlFHSYlvhRIL)



BigQuery ML has additional requirements: training AutoML models are supported in certain locations only.
Check your dateset location before starting to experiment!


[BqML and locations mapping](https://cloud.google.com/bigquery/docs/locations#bqml-loc)

More info about the model used. [link](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree#dropout)



If you get unsupported dataset location error [link](https://cloud.google.com/knowledge/kb/bigquery-unsupported-dataset-location-for-automl-jobs-000004465)


*Experiment predicts over created user datamart query by default*



In [ ]:
#@title Event propensity model inputs
Target_time_window_start_date= "2024-01-01" #@param {type:"date"}
Target_time_window_end_date= "2024-02-01" #@param {type:"date"}
Learning_window_start_date= "2023-10-01" #@param {type:"date"}
Target_event= "view_item" #@param {type:"string"}

### Generate the modeling queries

In [ ]:
#adjust learning and target time frames

target_start_table_suffix = reformat_date(Target_time_window_start_date)
target_end_table_suffix = reformat_date(Target_time_window_end_date)
target_time_frame = f"(_TABLE_SUFFIX BETWEEN '{target_start_table_suffix}' AND '{target_end_table_suffix}')"

flagging_query= f"""
create or replace table `{Big_query_project_name}.{Data_set_name}.user_flags_{target_start_table_suffix}_{target_end_table_suffix}` as
SELECT
u1.user_id,
max(case when event_name = '{Target_event}' then 1 else 0 end) as target_{Target_event},
FROM `{Big_query_project_name}.{GA4_table_name}.events_*` as x
left join `{Big_query_project_name}.{Data_set_name}.user_mapping_table` as u1 on x.user_pseudo_id = u1.user_pseudo_id where {target_time_frame} and u1.user_id is not null
group by 1;"""

Target_start_datetime_object = pd.to_datetime(Target_time_window_start_date)

#define training window
# Subtract 1 day from the target start date. Create the end date of learning period. learning period should end one day prior to target period.
Target_start_datetime_object_minus_one_day = Target_start_datetime_object - pd.DateOffset(days=1)
Target_start_datetime_object_minus_one_day = Target_start_datetime_object_minus_one_day.date()


features_start_date_table_suffix = reformat_date(Learning_window_start_date)
features_end_date_table_suffix = reformat_date(str(Target_start_datetime_object_minus_one_day))

learning_time_frame = f"(_TABLE_SUFFIX BETWEEN '{features_start_date_table_suffix}' AND '{features_end_date_table_suffix}')"

#adjust the time frame for the training set and build training table
training_period_query = user_datamart_query.replace(time_frame, learning_time_frame)
training_period_query = training_period_query.replace(f"user_id_data_ga4_{table_suffix_start_date}_{table_suffix_end_date}", f"user_features_train_temp_{features_start_date_table_suffix}_{features_end_date_table_suffix}")


#outlier handling using winsorizing
winsor_columns = [
                  "number_of_page_views_7D",
                  "number_of_page_views_14D",
                  "number_of_page_views_30D",
                  "visit_freq_7D",
                  "visit_freq_14D",
                  "visit_freq_30D",
                  "visit_freq_60D",
                  "visit_duration_7Ds",
                  "visit_duration_14Ds",
                  "visit_duration_30Ds",
                  "visit_duration_60Ds",
                  "total_revenue_7D",
                  "total_revenue_14D",
                  "total_revenue_30D",
                  "total_revenue_60D",
                  "purchase_recency",
                  "purchase_events_30D",
                  "purchase_events_14D",
                  "purchase_events_7D",
                  "total_clicks_7D",
                  "total_clicks_14D",
                  "total_clicks_30D"]

winsor_query_0 = ""

for column_name in winsor_columns:
  winsor_template = f"""IF({column_name}> (PERCENTILE_CONT({column_name}, 0.97) OVER ()), round(PERCENTILE_CONT({column_name}, 0.97) OVER (),2), {column_name}) as {column_name},"""
  winsor_query_0 += winsor_template + '\n'

# Remove the trailing comma from the last line
winsor_query_0 = winsor_query_0.rstrip(',\n')

#complete winsorizin query
winsor_query = sql_query = f"""create or replace table `{Big_query_project_name}.{Data_set_name}.user_features_train_{features_start_date_table_suffix}_{features_end_date_table_suffix}` as
SELECT
    * EXCEPT({', '.join(winsor_columns)}),
    {winsor_query_0}
FROM
    `{Big_query_project_name}.{Data_set_name}.user_features_train_temp_{features_start_date_table_suffix}_{features_end_date_table_suffix}`;
"""



#undersampling query and joining flagged users with training features
undersampling_query = f"""create or replace table `{Big_query_project_name}.{Data_set_name}.train_set_{Target_event}` as
with train1 as
(SELECT
t1.user_id,
target_{Target_event},
t2.* except(user_id)
 FROM `{Big_query_project_name}.{Data_set_name}.user_flags_{target_start_table_suffix}_{target_end_table_suffix}` t1
 inner join `{Big_query_project_name}.{Data_set_name}.user_features_train_{features_start_date_table_suffix}_{features_end_date_table_suffix}` t2 on t1.user_id = t2.user_id
)
,
PositiveSamples AS (
  SELECT *
  FROM train1
  WHERE target_{Target_event} = 1
)
, NegativeSamples AS (
  SELECT *,
         ROW_NUMBER() OVER (ORDER BY RAND()) as row_num
  FROM train1
  WHERE target_{Target_event} = 0
)

SELECT *
FROM PositiveSamples
UNION ALL
SELECT *except(row_num)
FROM NegativeSamples
WHERE row_num <= (SELECT COUNT(*) FROM PositiveSamples);"""


# training the model boosted tree classifier
model_query = f"""
create or replace model `{Big_query_project_name}.{Data_set_name}.my_model`
options(
  MODEL_TYPE = 'BOOSTED_TREE_CLASSIFIER',
  CATEGORY_ENCODING_METHOD = 'LABEL_ENCODING',
  ENABLE_GLOBAL_EXPLAIN = TRUE,
  DATA_SPLIT_METHOD = 'AUTO_SPLIT',
  INPUT_LABEL_COLS = ['target_{Target_event}']
)

as

SELECT *except(user_id,crtdate, last_event_date) FROM `{Big_query_project_name}.{Data_set_name}.train_set_{Target_event}`;
"""

clean_up_query = f"""drop table `{Big_query_project_name}.{Data_set_name}.user_features_train_temp_{features_start_date_table_suffix}_{features_end_date_table_suffix}`;
drop table `{Big_query_project_name}.{Data_set_name}.user_features_train_{features_start_date_table_suffix}_{features_end_date_table_suffix}`;
drop table `{Big_query_project_name}.{Data_set_name}.user_flags_{target_start_table_suffix}_{target_end_table_suffix}`;"""

training_query = flagging_query + '\n' + training_period_query + '\n' + winsor_query + '\n' + undersampling_query + '\n' + model_query + '\n' + clean_up_query

training_query_name = "model_training_query.sql"

# Write the SQL query to the file
with open(training_query_name, "w") as file:
    file.write(training_query)



prediction_query= f"""create or replace table `{Big_query_project_name}.{Data_set_name}.predictions_by_customer_target_{Target_event}` as
SELECT
  user_id,
  predicted_target_{Target_event},
  predicted_target_{Target_event}_probs
FROM
  ML.PREDICT(MODEL `{Big_query_project_name}.{Data_set_name}.my_model`,
    (
    SELECT
  *
    FROM
      `{Big_query_project_name}.{Data_set_name}.user_id_data_ga4_{table_suffix_start_date}_{table_suffix_end_date}`));"""


prediction_query_name = "model_prediction_query.sql"  # You can use .sql extension to indicate that it is an SQL file

# Write the SQL query to the file
with open(prediction_query_name, "w") as file:
    file.write(prediction_query)

### Create tables in Bigquery

In [ ]:
queryjob4= client.query(training_query)
queryjob4.result()

queryjob5 = client.query(prediction_query)
queryjob5.result()